In [21]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle 

In [22]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [23]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [24]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

86 documents
22 classes ['Cash', 'Coconut', 'Cotton', 'Food', 'Horticulture', 'Jute', 'Maize', 'Millets', 'Oilseeds', 'Plantation', 'Pulses', 'Rice', 'Rubber', 'Sugarcane', 'Tea', 'Tobacco', 'Wheat', 'crop_types', 'goodbye', 'greeting', 'options', 'thanks']
69 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'cash', 'chatting', 'coconut', 'cotton', 'could', 'crop', 'day', 'detail', 'different', 'do', 'example', 'fertilizer', 'food', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'horticulture', 'how', 'indian', 'is', 'jute', 'later', 'list', 'maize', 'me', 'millet', 'next', 'nice', 'of', 'offered', 'oilseed', 'plantation', 'producer', 'provide', 'pulse', 'rice', 'rubber', 'see', 'sugarcane', 'support', 'tea', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'tobacco', 'type', 'used', 'what', 'wheat', 'you']
Training data created


C:\Users\Trishita\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
86/86 [==============================] - 0s 1ms/step - loss: 3.1073 - accuracy: 0.1047
Epoch 2/200
86/86 [==============================] - 0s 310us/step - loss: 3.0602 - accuracy: 0.0698
Epoch 3/200
86/86 [==============================] - 0s 402us/step - loss: 2.9665 - accuracy: 0.1395
Epoch 4/200
86/86 [==============================] - 0s 377us/step - loss: 2.9454 - accuracy: 0.0930
Epoch 5/200
86/86 [==============================] - 0s 394us/step - loss: 2.8845 - accuracy: 0.1860
Epoch 6/200
86/86 [==============================] - 0s 411us/step - loss: 2.8009 - accuracy: 0.1744
Epoch 7/200
86/86 [==============================] - 0s 367us/step - loss: 2.6985 - accuracy: 0.3023
Epoch 8/200
86/86 [==============================] - 0s 404us/step - loss: 2.6290 - accuracy: 0.1977
Epoch 9/200
86/86 [==============================] - 0s 383us/step - loss: 2.3881 - accuracy: 0.2791
Epoch 10/200
86/86 [==============================] - 0s 382us/step - loss: 2.3186 - accuracy